# Tratamento de Dados - Leads Novos

In [1]:
import gspread
import numpy as np
import os
import pandas as pd
import pytz
import re
import requests
import unicodedata

from datetime import datetime
from dotenv import load_dotenv
from google.oauth2 import service_account
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from pathlib import Path

# === Carregar variáveis de ambiente ===
load_dotenv(Path.cwd().parent / "secrets" / ".env", override=True)

# === Definir escopos de acesso ===
scopes = [
    "https://www.googleapis.com/auth/spreadsheets.readonly",
    "https://www.googleapis.com/auth/drive.readonly"
]

# === Caminho seguro para o arquivo JSON (lido do .env) ===
cred_path = Path.cwd().parent / os.getenv("GOOGLE_CREDENTIALS_PATH")

# === Carregar credenciais de forma moderna ===
creds = service_account.Credentials.from_service_account_file(
    str(cred_path),
    scopes=scopes
)

# === Autorizar o cliente gspread ===
client = gspread.authorize(creds)

# === Função para carregar aba da planilha ===
def carregar_aba(sheet_id, aba_nome):
    planilha = client.open_by_key(sheet_id)
    aba = planilha.worksheet(aba_nome)
    dados = aba.get_all_records()
    return pd.DataFrame(dados)

# === IDs das planilhas (fixar no .env depois se quiser) ===
id_leads_l34 = "155-8YH18_ExLOLq0kWQTz1rolB4903hV6_MfD1FubkQ"
id_invest_trafego_l34 = "1A7W3qqCjDDP8oDpjLt6hZHLs68a1wmD1kWn47StFOTE"

# === Carregar DataFrames ===
df_leads_l34_forms = carregar_aba(id_leads_l34, "Respostas ao formulário 1")
df_dados_invest_face = carregar_aba(id_invest_trafego_l34, "Dados Investimento")
df_dados_invest_google = carregar_aba(id_invest_trafego_l34, "Dados Investimento Google")

# Pesquisa Leads Forms

In [2]:
df_leads_l34_forms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2101 entries, 0 to 2100
Data columns (total 14 columns):
 #   Column                                                                                                                                    Non-Null Count  Dtype 
---  ------                                                                                                                                    --------------  ----- 
 0   Carimbo de data/hora                                                                                                                      2101 non-null   object
 1   Nome completo                                                                                                                             2101 non-null   object
 2   Seu melhor e-mail                                                                                                                         2101 non-null   object
 3   Número de WhatsApp                                        

In [3]:
df_leads_l34_forms.drop(columns=[
    "O que você espera aprender/ver no evento Operação Policial Civil - 2025? ",
    "Imagine que você está em uma mentoria individual com o Prof. Luis Costa, quais seriam as 3 principais perguntas que você faria para ele?"
], inplace=True)

In [4]:
# Renomear colunas do df_leads_google
colunas_renomeadas = {
    'Carimbo de data/hora': 'data',
    'Seu melhor e-mail': 'email',
    'Nome completo': 'nome',
    'Número de WhatsApp': 'whatsapp',
    'Qual estado você reside?': 'estado',
    'Qual sua faixa de idade?': 'idade',
    'Qual seu nível de escolaridade?': 'escolaridade',
    'Qual sua faixa salarial atualmente?': 'renda',
    'Qual seu estado civil?': 'estado_civil',
    'Você tem filhos?': 'filhos',
    'Por que você escolheu/escolheria a profissão de Policial Civil?': 'escolheu_profissao',
    'O que está IMPEDINDO você de conquistar o seu DISTINTIVO e ser o próximo POLICIAL CIVIL? Qual sua maior dificuldade?': 'dificuldade'
}

df_leads_l34_forms.rename(columns=colunas_renomeadas, inplace=True)

# Normalizar e deduplicar e-mails
if 'email' in df_leads_l34_forms.columns:
    df_leads_l34_forms['email'] = df_leads_l34_forms['email'].astype(str).str.lower().str.strip()

# Exibir resumo
df_leads_l34_forms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2101 entries, 0 to 2100
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   data                2101 non-null   object
 1   nome                2101 non-null   object
 2   email               2101 non-null   object
 3   whatsapp            2101 non-null   object
 4   estado              2101 non-null   object
 5   idade               2101 non-null   object
 6   escolaridade        2101 non-null   object
 7   renda               2101 non-null   object
 8   estado_civil        2101 non-null   object
 9   filhos              2101 non-null   object
 10  escolheu_profissao  2101 non-null   object
 11  dificuldade         2101 non-null   object
dtypes: object(12)
memory usage: 197.1+ KB


In [5]:
df_leads_l34_forms["renda"] = (
    df_leads_l34_forms["renda"]
    .replace({
        "De R$ 1.000,00 a R$ 3.000,00": "De 1.000 a 3.000",
        "De R$ 3.000,00 a R$ 5.000,00": "De 3.000 a 5.000",
        "Até R$ 1.000,00": "Até 1.000",
        "Não estou trabalhando no momento": "Desempregado",
        "Acima de R$ 5.000,00": "Acima de 5.000"
    })
)

In [6]:
# Normalize o dicionário para garantir que as chaves estejam em minúsculo
estado_siglas = {
    k.lower(): v for k, v in {
        'são paulo': 'SP',
        'sp': 'SP',
        'rio de janeiro': 'RJ',
        'rj': 'RJ',
        'minas gerais': 'MG',
        'mg': 'MG',
        'bahia': 'BA',
        'ba': 'BA',
        'goiás': 'GO',
        'goias': 'GO',
        'go': 'GO',
        'paraná': 'PR',
        'pr': 'PR',
        'espírito santo': 'ES',
        'es': 'ES',
        'rio grande do sul': 'RS',
        'rs': 'RS',
        'santa catarina': 'SC',
        'sc': 'SC',
        'alagoas': 'AL',
        'al': 'AL',
        'paraíba': 'PB',
        'paraiba': 'PB',
        'pb': 'PB',
        'pará': 'PA',
        'pa': 'PA',
        'maranhão': 'MA',
        'ma': 'MA',
        'pernambuco': 'PE',
        'pe': 'PE',
        'amazonas': 'AM',
        'am': 'AM',
        'rio grande do norte': 'RN',
        'rn': 'RN',
        'distrito federal': 'DF',
        'df': 'DF',
        'boa vista': 'RR',
        'rr': 'RR',
        'ceará': 'CE',
        'brasília': 'DF',
        'brasilia': 'DF',
        'florianopolis': 'SC',
        'guarulhos': 'SP',
        'guarujá': 'SP',
        'Rio das ostras': 'RJ',
        'Granja': 'RJ',
        'Ceara': 'CE',
        'Tocantins': 'TO',
        'Belém': 'PA',
        'Acre': 'AC',
        'Rondônia': 'RO',
        'Itanhaém': 'SP',
        'Rio de janriro': 'RJ',
        'Volta redonda': 'RJ',
        'Botucatu': 'SP',
        'Mogi mirim': 'SP',
        'Feira de Santana': 'BA',
        'Recife': 'PE',
        'Joinville': 'SC',
        'São Bento do Sul': 'SC',
        'Juazeiro do norte': 'CE'
    }.items()
}

# Função de normalização
def normalizar_estado(valor):
    if pd.isna(valor):
        return None
    texto = str(valor).lower().strip()
    for nome, sigla in estado_siglas.items():
        if nome in texto:
            return sigla
    return valor

# Aplicar a função
df_leads_l34_forms['estado'] = df_leads_l34_forms['estado'].apply(normalizar_estado)

# Verificar resultado final
print(df_leads_l34_forms['estado'].value_counts(dropna=False))

estado
SP                    1374
MG                     155
BA                      93
PR                      75
SC                      67
RS                      62
PE                      61
RJ                      58
GO                      28
AL                      25
MA                      25
CE                      24
DF                      18
ES                       6
PA                       6
RN                       5
AM                       2
Piauí                    1
Florianópolis            1
Piauí                    1
Vila Rica MT             1
Río grande do sul        1
Avaré                    1
Gandu                    1
Concórdia                1
PB                       1
CE                       1
Góiase                   1
Ce                       1
MT                       1
Jaraguá do sul           1
AC                       1
TO                       1
Ipu-ce                   1
Name: count, dtype: int64


In [7]:
# Categorias permitidas
CATEGORIAS_PERMITIDAS = {
    'médio completo',
    'superior completo',
    'fundamental completo',
    'superior incompleto',
    'técnico',
    'médio incompleto',
    'fundamental incompleto'
}

# 1. Pré-processamento
def limpar_texto(texto):
    texto = str(texto).lower().strip()
    return unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')

# 2. Substituições comuns
SUBSTITUICOES = {
    'imcompleto': 'incompleto',
    'inclopeto': 'incompleto',
    'incompleta': 'incompleto',
    'enfetmagem': 'enfermagem',
    'encino': 'ensino',
    'esino': 'ensino',
    'ensaio': 'ensino',
    'ensino superior incomoda': 'superior incompleto',
    'sup incompleto mais cursando': 'superior incompleto',
    'superior curso': 'superior incompleto',
    'superior engenharia mecanica incompleto': 'superior incompleto',
    'comecei direito, mas nao finalizei': 'superior incompleto',
    'faco radiologia': 'superior incompleto',
}

def aplicar_substituicoes(val):
    for k, v in SUBSTITUICOES.items():
        val = val.replace(k, v)
    return val

# 3. Mapeamento explícito
MAPEAMENTO_EXPLICITO = {
    'incompleto': 'superior incompleto',
    'none': None,
    'cursando': 'superior incompleto',
    'setimo ano': 'fundamental incompleto',
    'comecando a faculdade': 'superior incompleto',
    'cursando terceiro ano medio': 'médio incompleto',
    'bombeira civil': 'técnico',
    'sim': None,
    'auxiliar de enfermagem': 'técnico',
    'enfermagem': 'superior incompleto',
    '5 seria': 'fundamental incompleto',
    '8 seria': 'fundamental incompleto',
    'estudante': 'médio incompleto',
    'pos graduado': 'superior completo',
    'estou no 2°ano ensino medio': 'médio incompleto',
    'e completo': 'médio completo',
    'cursando pedagogia': 'superior incompleto',
    'cursando gestao em rh': 'superior incompleto',
    "2'ano": 'fundamental incompleto',
    'tec enfermagem': 'técnico',
    'ciencias contabeis': 'superior incompleto',
    'ensino medio completando': 'médio incompleto',
    '2': 'fundamental incompleto',
    'cursando o supervisor': 'superior incompleto',
    'estudar ainda': 'médio incompleto',
    'cursando gestao agronegocio': 'superior incompleto',
    'terapeuta ocupacional ( cursando)': 'superior incompleto',
    'engenharia nao concluida': 'superior incompleto',
    'preciso terminar o ensino fundamental': 'fundamental incompleto',
    'alguns certificados na area da seguranca': 'técnico',
}

def aplicar_mapeamento_explicito(val):
    return MAPEAMENTO_EXPLICITO.get(val)

# 4. Categorização padrão
def categorizar_padrao(val):
    if any(p in val for p in ['fundamental incompleto', 'parei no', 'nao terminei fundamental', 'nono ano', 'quint', 'sere', 'serie']) \
       or re.search(r'\b[1-9]{1,2}[ºo]?\s*(ano|serie)', val) and 'medio' not in val:
        return 'fundamental incompleto'
    
    if 'fundamental completo' in val or 'primeiro grau completo' in val:
        return 'fundamental completo'

    if any(p in val for p in ['ensino medio incompleto', 'medio incompleto', '2º colegial incompleto', 'eja', 'ensino medio nao']):
        return 'médio incompleto'

    if any(p in val for p in ['ensino medio completo', 'medio completo', 'magisterio']):
        return 'médio completo'

    if any(p in val for p in ['superior completo', 'graduacao completa', 'nivel superior completo', 'pos-graduacao', 'pos graduacao']) \
       or ('graduacao' in val and 'cursando' not in val and 'incompleto' not in val):
        return 'superior completo'

    if any(p in val for p in ['tecnico', 'tecnologo', 'tecnologia', 'curso tec']):
        return 'técnico'

    if any(v in val for v in ['cursando', 'concluindo', 'fazendo', 'em andamento', 'graduando', 'estudando', 'terminando']):
        if any(n in val for n in ['superior', 'faculdade', 'universidade', 'direito', 'biomedicina', 'administracao']):
            return 'superior incompleto'
        elif 'ensino medio' in val or 'medio' in val or 'colegial' in val:
            return 'médio incompleto'
        elif 'fundamental' in val or 'serie' in val:
            return 'fundamental incompleto'

    if re.search(r'[0-9]{1,2}.*semestre', val) or 'periodo' in val:
        return 'superior incompleto'

    return val

# 5. Ajustes finais
AJUSTES_FINAIS = {
    'pos graduado': 'superior completo',
    'pos graduada': 'superior completo',
    'pos-graduacao': 'superior completo',
    'pos-graduada': 'superior completo',
    'pos graduado em gestao de projetos': 'superior completo',
    'mestrado': 'superior completo',
    'mba': 'superior completo',
    'cursando graduacao': 'superior incompleto',
    'cursando graduacao em radiologia': 'superior incompleto',
    'cursando graduacao em enfermagem': 'superior incompleto',
    'cursanso ensino superior': 'superior incompleto',
    'cursanso faculdade': 'superior incompleto',
    'superior incompleto ☹️': 'superior incompleto',
    'estou, termiando o ensino medio!': 'médio incompleto',
    '8 incompleto': 'fundamental incompleto',
}

def aplicar_ajustes_finais(val):
    return AJUSTES_FINAIS.get(val, val)

# --------------------------
# Função principal
# --------------------------
def normalizar_escolaridade(valor):
    if pd.isna(valor):
        return None

    val = limpar_texto(valor)
    val = aplicar_substituicoes(val)

    mapeado = aplicar_mapeamento_explicito(val)
    if mapeado:
        val = mapeado
    else:
        val = categorizar_padrao(val)

    val = aplicar_ajustes_finais(val)

    if val not in CATEGORIAS_PERMITIDAS:
        return None

    return val

In [8]:
df_leads_l34_forms['escolaridade_original'] = df_leads_l34_forms['escolaridade']
df_leads_l34_forms['escolaridade'] = df_leads_l34_forms['escolaridade_original'].apply(normalizar_escolaridade)

df_leads_l34_forms['escolaridade'].value_counts()

escolaridade
médio completo            1404
superior completo          282
fundamental completo       252
superior incompleto         68
técnico                     23
médio incompleto            21
fundamental incompleto      16
Name: count, dtype: int64

In [9]:
# Função de limpeza de texto
def limpar_texto(texto):
    texto = str(texto).strip().lower()
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    texto = re.sub(r'[^\w\s]', '', texto)  # remove pontuação
    return texto

# Mapeamento por categoria
def normalizar_estado_civil(valor):
    if pd.isna(valor):
        return None

    val = limpar_texto(valor)

    if val in {'solteiroa', 'solteiro', 'viuvo solteiro'}:
        return 'solteiro(a)'

    if val in {'casadoa', 'tenho esposa e filhos mais nao casado formalmente'}:
        return 'casado(a)'

    if val in {
        'divorciadoa ou separadora', 'divorciadoa', 'separado', 'separada', 'em processo de separacao',
        'divorciando', 'divorciado uniao estavel'
    }:
        return 'divorciado(a)'

    if val in {
        'uniao estavel', 'moro junto', 'mora junto', 'moramos juntos', 'morando junto',
        'morando com alguem', 'convivente', 'amigado', 'amigada', 'amaziado',
        'moro com um pessoal', 'tenho companheira e filhos', 'moro a 20 anos',
        'moro a mais de 5 anos com minha conjuge', 'ajuntado sem uniao estavel', 'namorando'
    } or 'moro junto' in val or 'com meu parceiro' in val:
        return 'união estável'

    if val in {'viuva', 'viuvo', 'viuva de companheiro'}:
        return 'viúvo(a)'

    return valor  # mantém valor original se não reconhecido

# Ajustes finais de exceções residuais
AJUSTES_FINAIS_ESTADO_CIVIL = {
    'vendendor autonomo e sou vigilante': None,
    'estou terminado o ensino medio': None,
    '5serie': None,
    'enrolada': 'união estável',
    'leandro alves': None
}

def aplicar_ajustes_finais_estado_civil(valor):
    if valor in CATEGORIAS_VALIDAS_ESTADO_CIVIL:
        return valor  # já está padronizado

    val_limpo = limpar_texto(valor)
    return AJUSTES_FINAIS_ESTADO_CIVIL.get(val_limpo, None)  # se não reconhecido, descarta

# Categorias válidas
CATEGORIAS_VALIDAS_ESTADO_CIVIL = {
    'solteiro(a)',
    'casado(a)',
    'divorciado(a)',
    'união estável',
    'viúvo(a)'
}

In [10]:
df_leads_l34_forms['estado_civil_original'] = df_leads_l34_forms['estado_civil']
df_leads_l34_forms['estado_civil'] = df_leads_l34_forms['estado_civil_original'].apply(normalizar_estado_civil)

df_leads_l34_forms['estado_civil'].value_counts()

estado_civil
solteiro(a)                                         786
casado(a)                                           777
união estável                                       266
Divorciado(a) ou Separado(a)                        253
viúvo(a)                                             10
Amasiado                                              2
Tenho esposa mas não nO papel                         1
X                                                     1
Convivência                                           1
Morando junto com minha esposa a quase 11 anos        1
Morando junto mais nao no papel                       1
Divorciado porém atualmente moro com um cônjuge       1
Se conhecendo                                         1
Name: count, dtype: int64

In [11]:
# Categorias válidas para dificuldade
DIFICULDADES_VALIDAS = {
    'financeiro / dinheiro',
    'falta de tempo',
    'falta de base escolar',
    'falta de oportunidade',
    'idade'
}

# Função de limpeza e normalização
def limpar_dificuldade_simples(valor):
    if pd.isna(valor):
        return None

    val = str(valor).strip().lower().strip("., ")

    if val in DIFICULDADES_VALIDAS:
        return val
    return None

In [12]:
df_leads_l34_forms['dificuldade_original'] = df_leads_l34_forms['dificuldade']
df_leads_l34_forms['dificuldade'] = df_leads_l34_forms['dificuldade_original'].apply(limpar_dificuldade_simples)

df_leads_l34_forms['dificuldade'].value_counts()

dificuldade
financeiro / dinheiro    1018
falta de tempo            417
falta de base escolar     332
idade                     191
falta de oportunidade      20
Name: count, dtype: int64

In [13]:
# Excluir colunas que terminam com "_original"
df_leads_l34_forms.drop(
    columns=[col for col in df_leads_l34_forms.columns if col.endswith('_original')],
    inplace=True
)

In [14]:
df_leads_l34_forms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2101 entries, 0 to 2100
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   data                2101 non-null   object
 1   nome                2101 non-null   object
 2   email               2101 non-null   object
 3   whatsapp            2101 non-null   object
 4   estado              2101 non-null   object
 5   idade               2101 non-null   object
 6   escolaridade        2066 non-null   object
 7   renda               2101 non-null   object
 8   estado_civil        2101 non-null   object
 9   filhos              2101 non-null   object
 10  escolheu_profissao  2101 non-null   object
 11  dificuldade         1978 non-null   object
dtypes: object(12)
memory usage: 197.1+ KB


In [15]:
df_leads_l34_forms = df_leads_l34_forms.drop(columns=['nome'])

In [16]:
features = ["renda", "escolaridade", "idade", "estado_civil", "filhos", 'escolaridade', 'escolheu_profissao', 'dificuldade']

for col in features:
    if col in df_leads_l34_forms.columns:
        df_leads_l34_forms[col] = df_leads_l34_forms[col].astype(str).str.strip().str.lower()

In [17]:
colunas_excluir = ["data", "email", "whatsapp", "nome"]

for coluna in df_leads_l34_forms.columns:
    if coluna not in colunas_excluir:
        print(f"\nColuna: {coluna}")
        print(df_leads_l34_forms[coluna].value_counts(dropna=False))


Coluna: estado
estado
SP                    1374
MG                     155
BA                      93
PR                      75
SC                      67
RS                      62
PE                      61
RJ                      58
GO                      28
AL                      25
MA                      25
CE                      24
DF                      18
ES                       6
PA                       6
RN                       5
AM                       2
Piauí                    1
Florianópolis            1
Piauí                    1
Vila Rica MT             1
Río grande do sul        1
Avaré                    1
Gandu                    1
Concórdia                1
PB                       1
CE                       1
Góiase                   1
Ce                       1
MT                       1
Jaraguá do sul           1
AC                       1
TO                       1
Ipu-ce                   1
Name: count, dtype: int64

Coluna: idade
idade
36 - 45 anos

# Leads Novos

In [18]:
parquet_leads_antigos = Path.cwd().parent / "dados" / "leads_antigos.parquet"
parquet_alunos_antigos = Path.cwd().parent / "dados" / "alunos_antigos.parquet"
parquet_leads_l34 = Path.cwd().parent / "dados" / "leads_l34.parquet"

df_leads = pd.read_parquet(parquet_leads_antigos)
df_alunos = pd.read_parquet(parquet_alunos_antigos)
df_leads_l34 = pd.read_parquet(parquet_leads_l34)

In [19]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10863 entries, 0 to 10862
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   data                10863 non-null  datetime64[ns]
 1   lancamentos         10863 non-null  object        
 2   email               10863 non-null  object        
 3   whatsapp            10863 non-null  object        
 4   estado              10863 non-null  object        
 5   idade               10863 non-null  object        
 6   escolaridade        10863 non-null  object        
 7   renda               10863 non-null  object        
 8   estado_civil        10863 non-null  object        
 9   filhos              10863 non-null  object        
 10  escolheu_profissao  10863 non-null  object        
 11  dificuldade         10863 non-null  object        
 12  comprou             10863 non-null  int32         
 13  utm_source          10817 non-null  object    

In [20]:
df_alunos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   data                260 non-null    datetime64[ns]
 1   lancamentos         260 non-null    object        
 2   email               260 non-null    object        
 3   whatsapp            260 non-null    object        
 4   estado              260 non-null    object        
 5   idade               260 non-null    object        
 6   escolaridade        260 non-null    object        
 7   renda               260 non-null    object        
 8   estado_civil        260 non-null    object        
 9   filhos              260 non-null    object        
 10  escolheu_profissao  260 non-null    object        
 11  dificuldade         260 non-null    object        
 12  comprou             260 non-null    int32         
 13  utm_source          259 non-null    object        

In [21]:
df_leads_l34.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19011 entries, 0 to 19010
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   id                         19011 non-null  object
 1   nome                       19011 non-null  object
 2   email                      19011 non-null  object
 3   telefone                   19011 non-null  object
 4   data inscrição lançamento  19011 non-null  object
 5   estado                     15413 non-null  object
 6   idade                      15143 non-null  object
 7   escolaridade               15157 non-null  object
 8   renda                      15273 non-null  object
 9   estado civil               15217 non-null  object
 10  filhos                     15299 non-null  object
 11  escolheu profissão         15344 non-null  object
 12  dificuldade                14379 non-null  object
 13  email captação             15413 non-null  object
 14  telefo

In [22]:
taxa_resposta_por_canal = (
    df_leads_l34
    .groupby('utm_source')['email captação']
    .apply(lambda x: x.notna().mean())
    .to_dict()
)

In [23]:
display(taxa_resposta_por_canal)

{'EMAILMKT': 0.8333333333333334,
 'Facebook-Ads': 0.8065252658775461,
 'INSTAGRAM': 0.9244186046511628,
 'TIKTOK': 0.890625,
 'WHATSAPP': 0.8888888888888888,
 'YOUTUBE': 0.7868852459016393,
 'fb': 0.7777777777777778,
 'google': 0.8571428571428571,
 'google-ads': 0.8333333333333334,
 'search': 1.0,
 'teste': 0.0}

In [24]:
df_leads_l34 = df_leads_l34[df_leads_l34['email captação'].notna()].reset_index(drop=True)

In [25]:
df_leads_l34.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15413 entries, 0 to 15412
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   id                         15413 non-null  object
 1   nome                       15413 non-null  object
 2   email                      15413 non-null  object
 3   telefone                   15413 non-null  object
 4   data inscrição lançamento  15413 non-null  object
 5   estado                     15413 non-null  object
 6   idade                      15143 non-null  object
 7   escolaridade               15157 non-null  object
 8   renda                      15273 non-null  object
 9   estado civil               15217 non-null  object
 10  filhos                     15299 non-null  object
 11  escolheu profissão         15344 non-null  object
 12  dificuldade                14379 non-null  object
 13  email captação             15413 non-null  object
 14  telefo

In [26]:
df_leads_l34.isnull().sum()

id                              0
nome                            0
email                           0
telefone                        0
data inscrição lançamento       0
estado                          0
idade                         270
escolaridade                  256
renda                         140
estado civil                  196
filhos                        114
escolheu profissão             69
dificuldade                  1034
email captação                  0
telefone captação               2
utm_source                     15
utm_campaign                  420
utm_medium                    697
utm_content                   935
utm_term                     2267
dtype: int64

In [27]:
df_leads_l34 = df_leads_l34.drop(columns=['id', 'email', 'telefone', 'nome'])

colunas_renomeadas = {
    'data inscrição lançamento': 'data',
    'email captação': 'email',
    'telefone captação': 'whatsapp',
    'Qual estado você reside?': 'estado',
    'Qual sua faixa de idade?': 'idade',
    'Qual seu nível de escolaridade?': 'escolaridade',
    'Qual sua faixa salarial atualmente?': 'renda',
    'estado civil': 'estado_civil',
    'Você tem filhos?': 'filhos',
    'escolheu profissão': 'escolheu_profissao'
}

df_leads_l34.rename(columns=colunas_renomeadas, inplace=True)

df_leads_l34.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15413 entries, 0 to 15412
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   data                15413 non-null  object
 1   estado              15413 non-null  object
 2   idade               15143 non-null  object
 3   escolaridade        15157 non-null  object
 4   renda               15273 non-null  object
 5   estado_civil        15217 non-null  object
 6   filhos              15299 non-null  object
 7   escolheu_profissao  15344 non-null  object
 8   dificuldade         14379 non-null  object
 9   email               15413 non-null  object
 10  whatsapp            15411 non-null  object
 11  utm_source          15398 non-null  object
 12  utm_campaign        14993 non-null  object
 13  utm_medium          14716 non-null  object
 14  utm_content         14478 non-null  object
 15  utm_term            13146 non-null  object
dtypes: object(16)
memory u

In [28]:
df_leads_l34["renda"] = (
    df_leads_l34["renda"]
    .replace({
        "De R$ 1.000,00 a R$ 3.000,00": "De 1.000 a 3.000",
        "De R$ 3.000,00 a R$ 5.000,00": "De 3.000 a 5.000",
        "Até R$ 1.000,00": "Até 1.000",
        "Não estou trabalhando no momento": "Desempregado",
        "Acima de R$ 5.000,00": "Acima de 5.000"
    })
)

In [29]:
# Aplicar a função
df_leads_l34['estado'] = df_leads_l34['estado'].apply(normalizar_estado)

# Verificar resultado final
print(df_leads_l34['estado'].value_counts(dropna=False))

estado
SP       8889
Outro    3833
MG       1374
PR        760
RJ        458
MA         99
Name: count, dtype: int64


In [30]:
df_leads_l34['escolaridade_original'] = df_leads_l34['escolaridade']
df_leads_l34['escolaridade'] = df_leads_l34['escolaridade_original'].apply(normalizar_escolaridade)

df_leads_l34['escolaridade'].value_counts()

escolaridade
médio completo          10549
superior completo        2013
fundamental completo     1629
Name: count, dtype: int64

In [31]:
for col in features:
    if col in df_leads_l34.columns:
        df_leads_l34[col] = df_leads_l34[col].astype(str).str.strip().str.lower()

In [32]:
colunas_excluir = ["data", "email", "whatsapp"]

for coluna in df_leads_l34.columns:
    if coluna not in colunas_excluir:
        print(f"\nColuna: {coluna}")
        print(df_leads_l34[coluna].value_counts(dropna=False))


Coluna: estado
estado
SP       8889
Outro    3833
MG       1374
PR        760
RJ        458
MA         99
Name: count, dtype: int64

Coluna: idade
idade
36 - 45 anos        6412
26 - 35 anos        4443
46 - 55 anos        3131
até 25 anos         1033
none                 270
acima de 56 anos     124
Name: count, dtype: int64

Coluna: escolaridade
escolaridade
médio completo          10549
superior completo        2013
fundamental completo     1629
none                     1222
Name: count, dtype: int64

Coluna: renda
renda
de 1.000 a 3.000    7691
de 3.000 a 5.000    3870
até 1.000           1451
acima de 5.000      1148
desempregado        1113
none                 140
Name: count, dtype: int64

Coluna: estado_civil
estado_civil
solteiro(a)                     6074
casado(a)                       5329
união estável                   1752
divorciado(a) ou separado(a)    1729
outros                           333
none                             196
Name: count, dtype: int64

Coluna: 

# Concat Leads e Resposta Forms

In [33]:
df_leads_l34 = df_leads_l34.copy()
df_leads_l34_forms = df_leads_l34_forms.copy()

# Junte corretamente
df_leads_l34 = pd.concat([df_leads_l34, df_leads_l34_forms], ignore_index=True)
df_leads_l34.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17514 entries, 0 to 17513
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   data                   17514 non-null  object
 1   estado                 17514 non-null  object
 2   idade                  17514 non-null  object
 3   escolaridade           17514 non-null  object
 4   renda                  17514 non-null  object
 5   estado_civil           17514 non-null  object
 6   filhos                 17514 non-null  object
 7   escolheu_profissao     17514 non-null  object
 8   dificuldade            17514 non-null  object
 9   email                  17514 non-null  object
 10  whatsapp               17512 non-null  object
 11  utm_source             15398 non-null  object
 12  utm_campaign           14993 non-null  object
 13  utm_medium             14716 non-null  object
 14  utm_content            14478 non-null  object
 15  utm_term           

In [34]:
# Conta quantos e-mails aparecem mais de uma vez
duplicados = df_leads_l34['email'].duplicated(keep=False)

# Exibe o total de duplicados (incluindo todas as ocorrências)
total_duplicados = duplicados.sum()
print(f"Total de e-mails duplicados (todas ocorrências): {total_duplicados}")

Total de e-mails duplicados (todas ocorrências): 3364


In [35]:
emails_duplicados_unicos = df_leads_l34['email'][duplicados].nunique()
print(f"Total de e-mails diferentes que estão duplicados: {emails_duplicados_unicos}")

Total de e-mails diferentes que estão duplicados: 1660


In [36]:
# Normalizar e deduplicar e-mails
if 'email' in df_leads_l34.columns:
    df_leads_l34['email'] = df_leads_l34['email'].astype(str).str.lower().str.strip()

df_leads_l34 = df_leads_l34.drop_duplicates(subset='email', keep='first')

df_leads_l34["lancamentos"] = "L34"

df_leads_l34['data'] = pd.to_datetime(df_leads_l34['data'], errors='coerce')

df_leads_l34["comprou"] = df_leads_l34["email"].isin(df_alunos["email"]).astype(int)

df_leads_l34.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15776 entries, 0 to 17511
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   data                   15776 non-null  datetime64[ns]
 1   estado                 15776 non-null  object        
 2   idade                  15776 non-null  object        
 3   escolaridade           15776 non-null  object        
 4   renda                  15776 non-null  object        
 5   estado_civil           15776 non-null  object        
 6   filhos                 15776 non-null  object        
 7   escolheu_profissao     15776 non-null  object        
 8   dificuldade            15776 non-null  object        
 9   email                  15776 non-null  object        
 10  whatsapp               15774 non-null  object        
 11  utm_source             15398 non-null  object        
 12  utm_campaign           14993 non-null  object        
 13  utm_me

In [37]:
colunas_excluir = ["data", "email", "whatsapp"]

for coluna in df_leads_l34.columns:
    if coluna not in colunas_excluir:
        print(f"\nColuna: {coluna}")
        print(df_leads_l34[coluna].value_counts(dropna=False))


Coluna: estado
estado
SP       9114
Outro    3833
MG       1401
PR        777
RJ        469
MA        104
SC         13
BA         13
RS         13
PE         12
GO          8
CE          7
AL          6
PA          1
CE          1
PB          1
DF          1
RN          1
AM          1
Name: count, dtype: int64

Coluna: idade
idade
36 - 45 anos        6562
26 - 35 anos        4525
46 - 55 anos        3237
até 25 anos         1052
none                 270
acima de 56 anos     130
Name: count, dtype: int64

Coluna: escolaridade
escolaridade
médio completo            10787
superior completo          2057
fundamental completo       1680
none                       1227
superior incompleto          14
técnico                       4
fundamental incompleto        4
médio incompleto              3
Name: count, dtype: int64

Coluna: renda
renda
de 1.000 a 3.000    7877
de 3.000 a 5.000    3934
até 1.000           1509
acima de 5.000      1163
desempregado        1153
none                 140


# Adicionar Valores de Investimento de Trafégo

In [38]:
df_dados_invest_face.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 898 entries, 0 to 897
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   date                       898 non-null    object
 1   campaign                   898 non-null    object
 2   adset_name                 898 non-null    object
 3   ad_name                    898 non-null    object
 4   spend                      898 non-null    int64 
 5   impressions                898 non-null    int64 
 6   actions_link_click         898 non-null    int64 
 7   actions_landing_page_view  898 non-null    int64 
dtypes: int64(4), object(4)
memory usage: 56.3+ KB


In [39]:
df_dados_invest_face["spend"].sum()

2889702

In [40]:
df_dados_invest_face["spend"].value_counts()

spend
0       40
1       30
3       20
2       19
4       13
        ..
996      1
947      1
1013     1
1364     1
407      1
Name: count, Length: 577, dtype: int64

In [41]:
df_dados_invest_google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         53 non-null     object
 1   campaign     53 non-null     object
 2   clicks       53 non-null     int64 
 3   spend        53 non-null     int64 
 4   impressions  53 non-null     int64 
 5   conversions  53 non-null     int64 
dtypes: int64(4), object(2)
memory usage: 2.6+ KB


In [42]:
df_dados_invest_google["spend"].sum()

9317788

In [43]:
# Normalizar colunas
df_dados_invest_face['ad_name'] = df_dados_invest_face['ad_name'].str.strip().str.lower()
df_leads_l34['utm_content'] = df_leads_l34['utm_content'].str.strip().str.lower()

# Agrupar leads por utm_content e puxar o valor mais comum de lancamentos
leads_por_anuncio = (
    df_leads_l34.groupby('utm_content')
    .agg(num_leads=('utm_content', 'count'),
         lancamentos=('lancamentos', lambda x: x.mode().iloc[0] if not x.mode().empty else None))
    .reset_index()
)
leads_por_anuncio.rename(columns={'utm_content': 'ad_name'}, inplace=True)

# Agrupar investimento
investimento_por_anuncio = df_dados_invest_face.groupby('ad_name')['spend'].sum().reset_index()

# Merge leads + investimento
df_cpl_face = pd.merge(investimento_por_anuncio, leads_por_anuncio, on='ad_name', how='left')

# Pegar utm_source dominante por anúncio
utm_por_anuncio = (
    df_leads_l34
    .groupby('utm_content')['utm_source']
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    .reset_index()
    .rename(columns={'utm_content': 'ad_name'})
)

# Merge com df_cpl_face
df_cpl_face = pd.merge(df_cpl_face, utm_por_anuncio, on='ad_name', how='left')

# Agora sim!
df_cpl_face['spend'] = (df_cpl_face['spend'] / 100) * df_cpl_face['utm_source'].map(taxa_resposta_por_canal)

# Preencher NaNs
df_cpl_face['num_leads'] = df_cpl_face['num_leads'].fillna(0)

# Calcular CPL
df_cpl_face['cpl'] = df_cpl_face.apply(
    lambda row: row['spend'] / row['num_leads'] if row['num_leads'] > 0 else None,
    axis=1
)

# Total row
total_row = pd.DataFrame([{
    'ad_name': 'TOTAL',
    'spend': float(df_cpl_face['spend'].sum()),
    'num_leads': float(df_cpl_face['num_leads'].sum()),
    'cpl': float(df_cpl_face['spend'].sum()) / float(df_cpl_face['num_leads'].sum())
        if df_cpl_face['num_leads'].sum() > 0 else None,
    'lancamentos': None
}])

# Concatenar
df_cpl_face = pd.concat([df_cpl_face, total_row], ignore_index=True)

In [44]:
# Reordenar colunas colocando 'lancamentos' primeiro
cols = ['lancamentos'] + [col for col in df_cpl_face.columns if col != 'lancamentos']
df_cpl_face = df_cpl_face[cols]

# Renomear colunas
df_cpl_face.rename(columns={
    'ad_name': 'criativo',
    'spend': 'investimento',
    'num_leads': 'leads'
}, inplace=True)

# Ajustar formatos
df_cpl_face['leads'] = df_cpl_face['leads'].astype(int)
df_cpl_face['cpl'] = df_cpl_face['cpl'].round(2)
df_cpl_face['investimento'] = df_cpl_face['investimento'].round(2)  # opcional

pd.set_option('display.max_rows', 100)  # ou mais, se quiser
display(df_cpl_face)

,lancamentos,criativo,investimento,leads,utm_source,cpl
0,NaN,ads 14 - calendário v2 - vertical,NaN,0,NaN,NaN
1,NaN,ads 16 - carteira v2 - vertical,NaN,0,NaN,NaN
2,NaN,ads 18 - quanto pesa um distintivo v2 - vertical,NaN,0,NaN,NaN
3,NaN,ads13-cap-1080x1920-l34-opc3.0,NaN,0,NaN,NaN
4,NaN,ads15-cap-1080x1920-l34-opc3.0,NaN,0,NaN,NaN
5,L34,ads_001_captacao_video_stories_p1_imagem_sobral_1,7463.89,3902,Facebook-Ads,1.91
6,L34,ads_002_captacao_video_stories_p1_imagem_raiz_1,649.45,230,Facebook-Ads,2.82
7,L34,ads_003_captacao_video_stories_p1_video_mais_u...,1342.85,453,Facebook-Ads,2.96
8,L34,ads_004_captacao_video_stories_p1_video_4motiv...,1156.31,539,Facebook-Ads,2.15
9,L34,ads_005_captacao_video_stories_p1_video_vemai_v1,6374.91,3480,Facebook-Ads,1.83


In [45]:
df_cpl_face.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lancamentos   20 non-null     object 
 1   criativo      26 non-null     object 
 2   investimento  21 non-null     float64
 3   leads         26 non-null     int32  
 4   utm_source    20 non-null     object 
 5   cpl           21 non-null     float64
dtypes: float64(2), int32(1), object(3)
memory usage: 1.2+ KB


In [46]:
# Limpar e normalizar campanhas do Google Ads
df_dados_invest_google['campaign'] = df_dados_invest_google['campaign'].str.strip().str.lower()

# ➕ Normalizar categorias específicas para facilitar o merge
df_dados_invest_google['campaign'] = (
    df_dados_invest_google['campaign']
    .replace({
        r'.*search.*': 'search',
        r'.*pmax.*': 'pmax',
        r'.*tva.*': 'tvfa'
    }, regex=True)
)

# Leads: apenas limpar, sem normalizar nomes
df_leads_l34['utm_campaign'] = df_leads_l34['utm_campaign'].str.strip().str.lower()

In [47]:
# Agrupar leads por utm_content e puxar o valor mais comum de lancamentos
leads_por_anuncio = (
    df_leads_l34.groupby('utm_campaign')
    .agg(num_leads=('utm_campaign', 'count'),
         lancamentos=('lancamentos', lambda x: x.mode().iloc[0] if not x.mode().empty else None))
    .reset_index()
)
leads_por_anuncio.rename(columns={'utm_campaign': 'campaign'}, inplace=True)

df_dados_invest_google['spend'] = df_dados_invest_google['spend'].astype(str)

# Corrigir valores numéricos de spend com vírgula como decimal e ponto como milhar
df_dados_invest_google['spend'] = (
    df_dados_invest_google['spend']
    .str.replace('.', '', regex=False)   # remove milhar
    .str.replace(',', '.', regex=False)  # troca decimal
    .astype(float)
)

# Agrupar investimento
investimento_por_anuncio = df_dados_invest_google.groupby('campaign')['spend'].sum().reset_index()

# Merge leads + investimento
df_cpl_google = pd.merge(investimento_por_anuncio, leads_por_anuncio, on='campaign', how='left')

# Pegar utm_source dominante por anúncio
utm_por_anuncio = (
    df_leads_l34
    .groupby('utm_campaign')['utm_source']
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    .reset_index()
    .rename(columns={'utm_campaign': 'campaign'})
)

# Merge com df_cpl_face
df_cpl_google = pd.merge(df_cpl_google, utm_por_anuncio, on='campaign', how='left')

# Agora sim!
df_cpl_google['spend'] = (df_cpl_google['spend'] / 1000) * df_cpl_google['utm_source'].map(taxa_resposta_por_canal)

# Preencher NaNs
df_cpl_google['num_leads'] = df_cpl_google['num_leads'].fillna(0)

# Calcular CPL
df_cpl_google['cpl'] = df_cpl_google.apply(
    lambda row: row['spend'] / row['num_leads'] if row['num_leads'] > 0 else None,
    axis=1
)

# Total row
total_row = pd.DataFrame([{
    'campaign': 'TOTAL',
    'spend': float(df_cpl_google['spend'].sum()),
    'num_leads': float(df_cpl_google['num_leads'].sum()),
    'cpl': float(df_cpl_google['spend'].sum()) / float(df_cpl_google['num_leads'].sum())
        if df_cpl_google['num_leads'].sum() > 0 else None,
    'lancamentos': None
}])

# Concatenar
df_cpl_google = pd.concat([df_cpl_google, total_row], ignore_index=True)

In [48]:
# Reordenar colunas colocando 'lancamentos' primeiro
cols = ['lancamentos'] + [col for col in df_cpl_google.columns if col != 'lancamentos']
df_cpl_google = df_cpl_google[cols]

# Renomear colunas
df_cpl_google.rename(columns={
    'campaign': 'campanha',
    'spend': 'investimento',
    'num_leads': 'leads'
}, inplace=True)

# Ajustar formatos
df_cpl_google['leads'] = df_cpl_google['leads'].astype(int)
df_cpl_google['cpl'] = df_cpl_google['cpl'].round(2)
df_cpl_google['investimento'] = df_cpl_google['investimento'].round(2) 

pd.set_option('display.max_rows', 100)  # ou mais, se quiser
display(df_cpl_google)

,lancamentos,campanha,investimento,leads,utm_source,cpl
0,L34,pmax,1117.43,281,google-ads,3.98
1,L34,search,1015.02,107,google-ads,9.49
2,L34,tvfa,5632.37,1055,google-ads,5.34
3,None,TOTAL,7764.82,1443,NaN,5.38


# Concat `df_leads` com `df_lead_novo`

In [49]:
# Recarregue os dois DataFrames originais se possível (ou copie)
df_leads = df_leads.copy()
df_leads_l34 = df_leads_l34.copy()

# Junte corretamente
df_leads = pd.concat([df_leads, df_leads_l34], ignore_index=True)
print("Total:", len(df_leads))

Total: 26639


In [50]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26639 entries, 0 to 26638
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   data                   26639 non-null  datetime64[ns]
 1   lancamentos            26639 non-null  object        
 2   email                  26639 non-null  object        
 3   whatsapp               26637 non-null  object        
 4   estado                 26639 non-null  object        
 5   idade                  26639 non-null  object        
 6   escolaridade           26639 non-null  object        
 7   renda                  26639 non-null  object        
 8   estado_civil           26639 non-null  object        
 9   filhos                 26639 non-null  object        
 10  escolheu_profissao     26639 non-null  object        
 11  dificuldade            26639 non-null  object        
 12  comprou                26639 non-null  int32         
 13  u

In [51]:
colunas_excluir = ["data", "email", "whatsapp"]

for coluna in df_leads.columns:
    if coluna not in colunas_excluir:
        print(f"\nColuna: {coluna}")
        print(df_leads[coluna].value_counts(dropna=False))


Coluna: lancamentos
lancamentos
L34    15776
L32     4875
L31     2657
L33     2461
L30      352
L28      275
L29      243
Name: count, dtype: int64

Coluna: estado
estado
SP       17120
Outro     4049
MG        2436
RJ        1962
PR         818
MA         139
BA          26
RS          17
SC          16
GO          16
PE          13
CE           9
AL           8
DF           2
RN           2
AM           2
RR           1
PB           1
PA           1
CE           1
Name: count, dtype: int64

Coluna: idade
idade
36 - 45 anos        10746
26 - 35 anos         7895
46 - 55 anos         5585
até 25 anos          1830
acima de 56 anos      313
none                  270
Name: count, dtype: int64

Coluna: escolaridade
escolaridade
médio completo            18750
superior completo          3907
fundamental completo       2601
none                       1227
superior incompleto          94
técnico                      29
médio incompleto             16
fundamental incompleto       15
Name: c

In [52]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26639 entries, 0 to 26638
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   data                   26639 non-null  datetime64[ns]
 1   lancamentos            26639 non-null  object        
 2   email                  26639 non-null  object        
 3   whatsapp               26637 non-null  object        
 4   estado                 26639 non-null  object        
 5   idade                  26639 non-null  object        
 6   escolaridade           26639 non-null  object        
 7   renda                  26639 non-null  object        
 8   estado_civil           26639 non-null  object        
 9   filhos                 26639 non-null  object        
 10  escolheu_profissao     26639 non-null  object        
 11  dificuldade            26639 non-null  object        
 12  comprou                26639 non-null  int32         
 13  u

In [53]:
df_leads['whatsapp'] = df_leads['whatsapp'].astype(str)
df_alunos['whatsapp'] = df_alunos['whatsapp'].astype(str)

In [54]:
output_dir = Path.cwd().parent / "dados"

df_leads.to_parquet(output_dir / "leads.parquet", index=False)
df_alunos.to_parquet(output_dir / "alunos.parquet", index=False)
df_dados_invest_face.to_parquet(output_dir / "invest_face.parquet", index=False)
df_dados_invest_google.to_parquet(output_dir / "invest_google.parquet", index=False)

In [55]:
output_path = Path.cwd().parent / "dados"
df_cpl_face.to_parquet(output_path / "invest_trafego_face.parquet", index=False)
df_cpl_google.to_parquet(output_path / "invest_trafego_google.parquet", index=False)

In [56]:
# Gerar timestamp com fuso horário de SP
fuso_brasil = pytz.timezone("America/Sao_Paulo")
agora_brasil = datetime.now(fuso_brasil)

# Caminho seguro para salvar
config_path = Path.cwd().parent / "config" / "ultima_atualizacao.txt"
with open(config_path, "w") as f:
    f.write(agora_brasil.strftime("%Y-%m-%d %H:%M:%S"))

In [57]:
try:
    caminho_data = Path.cwd().parent / "config" / "ultima_atualizacao.txt"
    with open(caminho_data, "r") as f:
        texto = f.read()
        data_atualizacao = datetime.strptime(texto, "%Y-%m-%d %H:%M:%S")
        data_atualizacao_formatada = data_atualizacao.strftime("%d/%m/%Y %H:%M")
except Exception:
    data_atualizacao_formatada = "Desconhecida"

In [58]:
f"**Última atualização:** {data_atualizacao_formatada}"

'**Última atualização:** 27/05/2025 11:20'